In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building and Deploying a Human-in-the-Loop LangGraph Application with Reasoning Engine on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine%2Ftutorial_langgraph_human_in_the_loop.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Xiaolong Yang](https://github.com/shawn-yang-google) |

## Overview

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service designed to help you build and deploy agent reasoning frameworks.  [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for constructing stateful, multi-actor applications with LLMs, enabling the creation of sophisticated agent and multi-agent workflows.

This notebook demonstrates how to build, deploy, and test a Human-in-the-Loop LangGraph application using [Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) on Vertex AI. You'll learn how to combine LangGraph's powerful workflow orchestration with the scalability of Vertex AI to build Human-in-the-Loop generative AI applications.

The previous [notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph_human_in_the_loop.ipynb) covered: Defining Tools, Defining a Router, Building a LangGraph Application, Local Testing, Deploying to Vertex AI, Remote Testing, and Cleaning Up Resources.

This notebook expands on those concepts and explores the following `Human-in-the-Loop` features:

- **Reviewing Tool Calls:** Implement human oversight after tool use, allowing for verification and correction of actions before proceeding.
- **Fetching State History:** Retrieve the complete execution history of the LangGraph application for auditing, analysis, and potential state reversion.
- **Time Travel:** Examine the state of the agent at a specific point in time to understand past decisions.
- **Replay:** Restart execution from a specific checkpoint without modifications to ensure consistent results.
- **Branching:** Create alternative execution paths based on a past state, enabling the agent to explore different possibilities or correct previous errors.

By the end of this notebook, you'll possess the skills to build and deploy customized Human-in-the-Loop generative AI applications using LangGraph, Reasoning Engine, and Vertex AI.

## Get started

### Install the Vertex AI SDK and Required Packages

In [1]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    git+https://github.com/yeesian/langchain-google-spanner-python.git --force-reinstall

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.6/432.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information and Initialize the Vertex AI SDK

Before using Vertex AI, ensure you have an existing Google Cloud project and have [enabled the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Refer to the documentation for more details on [setting up a project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "reasoning-engine-test-1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://reasoning-engine-test-1-bucket"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Building and Deploying a LangGraph App on Reasoning Engine

The following sections guide you through building and deploying a LangGraph application using Reasoning Engine on Vertex AI.

### Import Libraries

Import the required Python libraries. These libraries provide the necessary tools for interacting with LangGraph, Vertex AI, and other components of the application.

In [3]:
from langchain.load import load as langchain_load
from vertexai.preview import reasoning_engines

### Define Tools

Begin by defining the tools for your LangGraph application. You'll define a custom Python function that serves as a tool within our agentic application.

In this example, we'll create a simple tool that retrieves the exchange rate requested by the user.  In practice, you can define functions to interact with APIs, query databases, or perform any other tasks your agent might need to execute.

In [4]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

### Define [`Checkpointers`](https://langchain-ai.github.io/langgraph/concepts/persistence/)

In LangGraph, [memory is checkpointing/persistence](https://github.com/langchain-ai/langgraph/discussions/352#discussioncomment-9290376). Checkpointing saves the [state](https://langchain-ai.github.io/langgraph/concepts/low_level/#checkpointer-state) of the agent's execution at each node in the graph, which is crucial for: `Resuming execution`, `Debugging and Inspection`, and `Asynchronous Operations`.

LangGraph provides a Checkpointer [Interface](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-interface), defining methods for saving and loading the state. Several built-in checkpointers are available to implement this interface.

Next, you'll define the arguments for your LangGraph application's checkpointer and create a custom Python function to act as the checkpointer builder. In this case, we'll define a simple `Spanner` checkpointer that returns the state of the current instance, database, and project.

In [5]:
checkpointer_kwargs = {
    "instance_id": "my-instance",
    "database_id": "my-database",
    "project_id": PROJECT_ID,
}


def checkpointer_builder(instance_id, database_id, project_id, **kwargs):
    from langchain_google_spanner import SpannerCheckpointSaver

    checkpointer = SpannerCheckpointSaver(instance_id, database_id, project_id)
    with checkpointer.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS checkpoints")
        cur.execute("DROP TABLE IF EXISTS checkpoint_writes")
    checkpointer.setup()

    return checkpointer

Alternatively, you can define a simple `MemorySaver` checkpointer
```python
checkpointer_kwargs = None


def checkpointer_builder(**kwargs):
    from langgraph.checkpoint.memory import MemorySaver

    return MemorySaver()
```

### Define the Human-in-the-Loop LangGraph Application

Now, you'll integrate all the components to define your Human-in-the-Loop LangGraph application within Reasoning Engine.

This application will utilize the tools and checkpointer you've defined. LangGraph offers a powerful framework for structuring these interactions and leveraging the capabilities of LLMs.

In [6]:
agent = reasoning_engines.LanggraphAgent(
    model="gemini-1.5-pro-001",
    tools=[get_exchange_rate],
    model_kwargs={"temperature": 0, "max_retries": 6},
    checkpointer_kwargs=checkpointer_kwargs,
    checkpointer_builder=checkpointer_builder,
)

### Local Testing

This section covers local testing of your LangGraph application before deployment to ensure it behaves as expected.

In [7]:
agent.set_up()

In [8]:
inputs = {
    "messages": [
        ("user", "What is the exchange rate from US dollars to Swedish currency?")
    ]
}

In [10]:
response = agent.query(
    input=inputs,
    config={"configurable": {"thread_id": "synchronous-thread-id"}},
)

response["messages"][-1]["kwargs"]["content"]

{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.', 'type': 'string'}}, 'type': 'object'}}}
{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurte

'The exchange rate from US dollars to Swedish krona is 1 USD to 11.0191 SEK. \n'

You can also utilize [streaming](https://langchain-ai.github.io/langgraph/how-tos/stream-values/) mode to stream back the `values` of the graph, representing the full state after each node execution.

In [11]:
for state_values in agent.stream_query(
    input=inputs,
    stream_mode="values",
    config={"configurable": {"thread_id": "streaming-thread-values"}},
):
    print(state_values)

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the exchange rate from US dollars to Swedish currency?', 'type': 'human', 'id': '5326270e-cf3a-4ed1-92da-54fc5f481a1d'}}]}
{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be speci

Alternatively, you can stream back `updates` to the graph. These represent the changes to the state after each node is executed.

In [12]:
for state_updates in agent.stream_query(
    input=inputs,
    stream_mode="updates",
    config={"configurable": {"thread_id": "streaming-thread-updates"}},
):
    print(state_updates)

{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.', 'type': 'string'}}, 'type': 'object'}}}
{'agent': {'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'function

## Human-in-the-loop

### Reviewing Tool Calls

LangGraph's Human-in-the-Loop functionality provides various [use cases](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/#use-cases) for incorporating human intervention and oversight into agent workflows (state machines). This notebook focuses on the [Reviewing Tool Calls](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/review-tool-calls/) use case.

To achieve this, the agent needs to [interrupt](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/#interrupt) execution in the following scenarios:

* Before invoking the tool (when the LLM generates a tool call AI Message).
* After receiving a tool response.

In [56]:
response = agent.query(
    input=inputs,
    interrupt_before=["tools"],  # Before invoking the tool.
    interrupt_after=["tools"],  # After getting a tool message.
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.', 'type': 'string'}}, 'type': 'object'}}}
================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (6708584a-4b64-4dcb-bdab-1ba69f49e4bb)
 Call ID: 6708584a

The process was interrupted *before invoking the tool*.

After review, we assume the LLM-generated tool call (`AI Message`) is correct and proceed to resume execution.

In [57]:
response = agent.query(
    input=None,  # Resume (continue with the tool call AI Message).
    interrupt_before=["tools"],
    interrupt_after=["tools"],
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

================================= Tool Message =================================
Name: get_exchange_rate

{"amount": 1.0, "base": "USD", "date": "2025-01-29", "rates": {"SEK": 11.0191}}


The process is interrupted again *after receiving the tool message*.

Upon review, if the LLM-generated `Tool Message` appears correct, we can resume execution.

In [15]:
response = agent.query(
    input=None,  # Resume (continue with the Tool Message).
    interrupt_before=["tools"],
    interrupt_after=["tools"],
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.', 'type': 'string'}}, 'type': 'object'}}}
================================== Ai Message ==================================

The exchange rate from US dollars to Swedish krona is 1 USD to 11.0191 SEK.


### Fetching State History

You can fetch the state history by calling `.get_state_history`.

In [16]:
for state_snapshot in agent.get_state_history(
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
):
    if state_snapshot["metadata"]["step"] >= 0:
        print(f'step {state_snapshot["metadata"]["step"]}: {state_snapshot["config"]}')
        state_snapshot["values"]["messages"][-1].pretty_print()
        print("\n")

step 3: {'configurable': {'thread_id': 'human-in-the-loop-deepdive', 'checkpoint_ns': '', 'checkpoint_id': '1efde79a-a2ce-6029-8003-bf82e3772b59'}}
================================== Ai Message ==================================

The exchange rate from US dollars to Swedish krona is 1 USD to 11.0191 SEK.


step 2: {'configurable': {'thread_id': 'human-in-the-loop-deepdive', 'checkpoint_ns': '', 'checkpoint_id': '1efde79a-78b6-6de1-8002-33c49cfdf22a'}}
================================= Tool Message =================================
Name: get_exchange_rate

{"amount": 1.0, "base": "USD", "date": "2025-01-29", "rates": {"SEK": 11.0191}}


step 1: {'configurable': {'thread_id': 'human-in-the-loop-deepdive', 'checkpoint_ns': '', 'checkpoint_id': '1efde79a-4b6e-6b3d-8001-6952abd5a7fc'}}
================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (c500fced-96c5-4c76-996c-90a0ef64ba1d)
 Call ID: c500fced-96c5-4c76-996c-90a0ef64ba1d
  Args:
 

### Time Travel

LangGraph's [Time Travel](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/) demonstrates how to build a conversational agent with persistent memory, enabling human intervention to correct past actions.  Essentially, it "rewinds" the conversation to a previous state, allows for mistake correction, and permits the agent to continue from that corrected point.

You can "time travel" by calling `.get_state`. By default, the agent retrieves the `latest state`.

In [17]:
state = agent.get_state(
    config={
        "configurable": {
            "thread_id": "human-in-the-loop-deepdive",
        }
    }
)

print(f'step {state["metadata"]["step"]}: {state["config"]}')
state["values"]["messages"][-1].pretty_print()

step 3: {'configurable': {'thread_id': 'human-in-the-loop-deepdive', 'checkpoint_ns': '', 'checkpoint_id': '1efde79a-a2ce-6029-8003-bf82e3772b59'}}
================================== Ai Message ==================================

The exchange rate from US dollars to Swedish krona is 1 USD to 11.0191 SEK.


To retrieve an earlier state, you need to specify the `checkpoint_id` (and `checkpoint_ns`).

In [18]:
snapshot_config = {}
for state_snapshot in agent.get_state_history(
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
):
    if state_snapshot["metadata"]["step"] == 1:
        snapshot_config = state_snapshot["config"]
        break

snapshot_config

{'configurable': {'thread_id': 'human-in-the-loop-deepdive',
  'checkpoint_ns': '',
  'checkpoint_id': '1efde79a-4b6e-6b3d-8001-6952abd5a7fc'}}

In [19]:
state = agent.get_state(config=snapshot_config)
print(f'step {state["metadata"]["step"]}: {state["config"]}')
state["values"]["messages"][-1].pretty_print()

step 1: {'configurable': {'thread_id': 'human-in-the-loop-deepdive', 'checkpoint_ns': '', 'checkpoint_id': '1efde79a-4b6e-6b3d-8001-6952abd5a7fc'}}
================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (c500fced-96c5-4c76-996c-90a0ef64ba1d)
 Call ID: c500fced-96c5-4c76-996c-90a0ef64ba1d
  Args:
    currency_from: USD
    currency_to: SEK


In [20]:
state

{'values': {'messages': [HumanMessage(content='What is the exchange rate from US dollars to Swedish currency?', additional_kwargs={}, response_metadata={}, id='4bcebac2-ea30-483f-8fa4-3ebaaa3300da'),
   AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"currency_from": "USD", "currency_to": "SEK"}', 'name': 'get_exchange_rate'}}, response_metadata={'avg_logprobs': -1.0086337344826181e-07, 'finish_reason': 'STOP', 'is_blocked': False, 'safety_ratings': [{'blocked': False, 'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.15234375, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1484375}, {'blocked': False, 'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.2451171875, 'severity': 'HARM_SEVERITY_LOW', 'severity_score': 0.26953125}, {'blocked': False, 'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1826

### Replay

LangGraph's [Replay](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/#replay-a-state) feature allows you to resume or replay a conversation from any specific point in its history.

You can initiate a replay by passing the `state["config"]` back to the agent. Note that the execution resumes exactly where it was left off, executing a tool call.

In [21]:
state["config"]

{'configurable': {'thread_id': 'human-in-the-loop-deepdive',
  'checkpoint_ns': '',
  'checkpoint_id': '1efde79a-4b6e-6b3d-8001-6952abd5a7fc'}}

In [22]:
for state_values in agent.stream_query(
    input=None,  # resume
    stream_mode="values",
    config=state["config"],
):
    langchain_load(state_values["messages"][-1]).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (c500fced-96c5-4c76-996c-90a0ef64ba1d)
 Call ID: c500fced-96c5-4c76-996c-90a0ef64ba1d
  Args:
    currency_from: USD
    currency_to: SEK
================================= Tool Message =================================
Name: get_exchange_rate

{"amount": 1.0, "base": "USD", "date": "2025-01-29", "rates": {"SEK": 11.0191}}
{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, '

### Branching

LangGraph's [Branching](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/#branch-off-a-past-state) feature allows you to modify and re-run a LangGraph conversation from a specific point in its history (rather than just from the latest state).  This enables the agent to explore alternate trajectories or allows a user to "version control" changes in a workflow.

In this example, you will:
* Update the tool calls from a previous step.
* Call `.update_state` to rerun the step with the updated configuration.

In [23]:
last_message = state["values"]["messages"][-1]
print(last_message)
print(last_message.tool_calls)

content='' additional_kwargs={'function_call': {'arguments': '{"currency_from": "USD", "currency_to": "SEK"}', 'name': 'get_exchange_rate'}} response_metadata={'avg_logprobs': -1.0086337344826181e-07, 'finish_reason': 'STOP', 'is_blocked': False, 'safety_ratings': [{'blocked': False, 'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.15234375, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1484375}, {'blocked': False, 'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.2451171875, 'severity': 'HARM_SEVERITY_LOW', 'severity_score': 0.26953125}, {'blocked': False, 'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1826171875, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.126953125}, {'blocked': False, 'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.10693359375

Update the tool calls from the previous step.

In [24]:
last_message.tool_calls[0]["args"]["currency_date"] = "2024-09-01"
last_message.tool_calls

[{'name': 'get_exchange_rate',
  'args': {'currency_from': 'USD',
   'currency_to': 'SEK',
   'currency_date': '2024-09-01'},
  'id': 'c500fced-96c5-4c76-996c-90a0ef64ba1d',
  'type': 'tool_call'}]

Call `.update_state` to rerun the step with the updated configuration.

In [25]:
branch_config = agent.update_state(
    config=state["config"],
    values={"messages": [last_message]},  # the update we want to make
)
branch_config

{'configurable': {'thread_id': 'human-in-the-loop-deepdive',
  'checkpoint_ns': '',
  'checkpoint_id': '1efde7aa-cb29-69f8-8002-26b3c27f0570'}}

In [26]:
for state_values in agent.stream_query(
    input=None,  # resume
    stream_mode="values",
    config=branch_config,
):
    langchain_load(state_values["messages"][-1]).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (c500fced-96c5-4c76-996c-90a0ef64ba1d)
 Call ID: c500fced-96c5-4c76-996c-90a0ef64ba1d
  Args:
    currency_date: 2024-09-01
    currency_from: USD
    currency_to: SEK
================================= Tool Message =================================
Name: get_exchange_rate

{"amount": 1.0, "base": "USD", "date": "2024-08-30", "rates": {"SEK": 10.2241}}
{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" 

## Deploying the Agent

In [27]:
def non_ddl_checkpointer_builder(instance_id, database_id, project_id, **kwargs):
    from langchain_google_spanner import SpannerCheckpointSaver

    return SpannerCheckpointSaver(instance_id, database_id, project_id)


remote_agent = reasoning_engines.ReasoningEngine.create(
    reasoning_engines.LanggraphAgent(
        model="gemini-1.5-pro-001",
        tools=[get_exchange_rate],
        model_kwargs={"temperature": 0, "max_retries": 6},
        checkpointer_kwargs=checkpointer_kwargs,
        checkpointer_builder=non_ddl_checkpointer_builder,
        # enable_tracing=True, # https://github.com/Arize-ai/openinference/issues/1119
    ),
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]",
        "git+https://github.com/yeesian/langchain-google-spanner-python.git",  # Optional. Remove if you are not using the Spanner checkpointer
    ],
)

remote_agent

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket reasoning-engine-test-1-bucket
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning-engine-test-1-bucket/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning-engine-test-1-bucket/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning-engine-test-1-bucket/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/349765473021/locations/us-central1/reasoningEngines/6091030535088373760/operations/7439983864732188672
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/349765473021/locations/us-centr

resource name: projects/349765473021/locations/us-central1/reasoningEngines/6091030535088373760

## Querying the Remote Agent

**Prerequisite**: Follow the instructions at https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/set-up#service-agent to grant the "Cloud Spanner Database User" role to your Reasoning Engine's P6SA. You also need to do the same at https://console.cloud.google.com/spanner/instances/my-instance/databases/my-database/details/tables?project=[PROJECT_ID]. Please note that it might [take some time for the permissions to propagate](https://cloud.google.com/iam/docs/access-change-propagation). For reference:
* https://cloud.google.com/spanner/docs/reference/rest/v1/projects.instances.databases.sessions/create
* https://cloud.google.com/spanner/docs/iam#roles

Otherwise, the following cells will return empty responses, and the logs will display an error similar to: `PermissionDenied: 403 Caller is missing IAM permission spanner.sessions.create on resource`.

### Remote testing

In [28]:
for state_updates in remote_agent.stream_query(
    input=inputs,
    stream_mode="updates",
    config={"configurable": {"thread_id": "remote-streaming-thread-updates"}},
):
    print(state_updates)

{'agent': {'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.15234375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1484375}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.2451171875, 'blocked': False, 'severity': 'HARM_SEVERITY_LOW', 'severity_score': 0.26953125}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1826171875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.126953125}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label

(You can verify that the corresponding data appears in https://console.cloud.google.com/spanner/instances/my-instance/databases/my-database/tables/checkpoints/details/data?project=[PROJECT_ID])

In [29]:
for state_values in remote_agent.stream_query(
    input=inputs,
    stream_mode="values",
    config={"configurable": {"thread_id": "remote-human-in-the-loop-overall"}},
):
    print(state_values)

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the exchange rate from US dollars to Swedish currency?', 'type': 'human', 'id': 'd47ea4ef-512a-4197-ae2b-ba9c0b4c4adf'}}]}
{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the exchange rate from US dollars to Swedish currency?', 'type': 'human', 'id': 'd47ea4ef-512a-4197-ae2b-ba9c0b4c4adf'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.15234375, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 's

### Reviewing Tool Calls

In [33]:
response = remote_agent.query(
    input=inputs,
    interrupt_before=["tools"],  # Before invoking the tool.
    interrupt_after=["tools"],  # After getting a tool message.
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_exchange_rate (51970fa6-db35-48fb-83ac-6efe9256c790)
 Call ID: 51970fa6-db35-48fb-83ac-6efe9256c790
  Args:
    currency_from: USD
    currency_to: SEK


In [34]:
response = remote_agent.query(
    input=None,  # Resume (continue with the tool call AI Message).
    interrupt_before=["tools"],
    interrupt_after=["tools"],
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

================================= Tool Message =================================
Name: get_exchange_rate

{"amount": 1.0, "base": "USD", "date": "2025-01-29", "rates": {"SEK": 11.0191}}


In [35]:
response = agent.query(
    input=None,  # Resume (continue with the Tool Message).
    interrupt_before=["tools"],
    interrupt_after=["tools"],
    config={"configurable": {"thread_id": "human-in-the-loop-deepdive"}},
)
langchain_load(response["messages"][-1]).pretty_print()

{'type': 'function', 'function': {'name': 'get_exchange_rate', 'description': 'Retrieves the exchange rate between two currencies on a specified date. Uses the Frankfurter API (https://api.frankfurter.app/) to obtain\n exchange rate data.', 'parameters': {'properties': {'currency_from': {'default': 'USD', 'description': 'The base currency (3-letter currency code). Defaults to "USD" (US Dollar).', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'description': 'The target currency (3-letter currency code). Defaults to "EUR" (Euro).', 'type': 'string'}, 'currency_date': {'default': 'latest', 'description': 'The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.', 'type': 'string'}}, 'type': 'object'}}}
================================== Ai Message ==================================

The current exchange rate (as of January 29, 2025) is 1 USD to 11.0191 SEK.


## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Reasoning Engine instance to avoid any unexpected charges on your Google Cloud account.

In [54]:
remote_agent.delete()